DMML Assignment 1

Submitted by : Trishita and Sowmya

This file contains two implementations of the finding frequent itemsets that we tested for various cases. While both produced correct results, we observed that Code 2 performed faster than Code 1. Therefore, we chose Code 2 for this assignment.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Code 1:**
- **Purpose**: Finds frequent $ K $-itemsets from a dataset using a recursive approach to generate all possible $ K $-itemsets.
- **Helper Functions**:
  1. **`load_vocab(vocab_file)`**:
     - Input: Vocabulary file path.
     - Output: Dictionary mapping word IDs to words.
  2. **`load_docword(docword_file)`**:
     - Input: Document-word file path.
     - Output: Two dictionaries:
       - `doc_words`: Maps document IDs to sets of word IDs.
       - `word_counts`: Maps word IDs to their total counts across all documents.
  3. **`generate_k_itemsets(words, K, index, current_itemset)`**:
     - Input: List of words, target size $ K $, current index, and current itemset being built.
     - Output: List of all $ K $-itemsets generated recursively.
  4. **`find_frequent_itemsets(vocab_file, docword_file, K, F)`**:
     - Input: Vocabulary file, document-word file, $ K $ (itemset size), $ F $ (frequency threshold).
     - Output: Execution time and count of frequent $ K $-itemsets.
     - Workflow:
       - Filters frequent words based on $ F $.
       - Generates all $ K $-itemsets for each document using recursion.
       - Counts occurrences of each $ K $-itemset and filters by $ F $.

In [8]:
import time

# Load vocabulary (wordID → word mapping)
def load_vocab(vocab_file):
    vocab_dict = {}
    with open(vocab_file, "r") as file:
        for i, word in enumerate(file, start=1):
            vocab_dict[i] = word.strip()
    return vocab_dict

In [9]:
# Load document-word data
def load_docword(docword_file):
    doc_words = {}  # Dictionary {docID: set(words)}
    word_counts = {}  # Dictionary {wordID: count}

    with open(docword_file, "r") as file:
        lines = file.readlines()

    D, W, NNZ = map(int, lines[:3])  # Read metadata

    for line in lines[3:]:
        docID, wordID, count = map(int, line.split())

        if docID not in doc_words:
            doc_words[docID] = set()
        doc_words[docID].add(wordID)

        word_counts[wordID] = word_counts.get(wordID, 0) + count

    return doc_words, word_counts

In [10]:
# Recursive function to generate K-itemsets
def generate_k_itemsets(words, K, index=0, current_itemset=[]):
    if len(current_itemset) == K:
        return [tuple(current_itemset)]  # Base case: return the itemset

    if index >= len(words):
        return []  # Stop if out of words

    # Recursive case: pick the current word and move forward OR skip it
    return generate_k_itemsets(words, K, index + 1, current_itemset + [words[index]]) + generate_k_itemsets(words, K, index + 1, current_itemset)

In [11]:
def find_frequent_itemsets(vocab_file, docword_file, K, F): # Main function to find frequent itemsets
    print(f"\nFinding Frequent {K}-Itemsets with F={F}...")

    start_time = time.time()

    # Load data
    vocab_dict = load_vocab(vocab_file)
    doc_words, word_counts = load_docword(docword_file)

    # Step 1: Filter words that appear at least F times
    frequent_words = {wordID for wordID, count in word_counts.items() if count >= F}

    # Step 2: Process each document and generate K-itemsets
    itemset_counts = {}

    for words in doc_words.values():   # words - set
        valid_words = sorted(wordID for wordID in words if wordID in frequent_words)

        if len(valid_words) >= K:
            k_itemsets = generate_k_itemsets(valid_words, K)

            for itemset in k_itemsets:
                itemset_counts[itemset] = itemset_counts.get(itemset, 0) + 1

    # Step 3: Filter itemsets by frequency threshold
    frequent_itemsets = {
        tuple(vocab_dict[wordID] for wordID in itemset): count
        for itemset, count in itemset_counts.items()
        if count >= F
    }

    execution_time = time.time() - start_time

    # Print results
    print(f"Execution Time: {execution_time:.2f} seconds")
    print(f"Total Frequent {K}-Itemsets Found: {len(frequent_itemsets)}")

    if frequent_itemsets:
        print("\nFirst 5 Frequent K-itemsets:")
        for itemset, count in list(frequent_itemsets.items())[:5]:  # Show first 10 examples
            print(f"Itemset: {itemset}, Count: {count}")
    else:
        print("No frequent itemsets found.")

    return execution_time, len(frequent_itemsets)

Test runs with KOS Dataset


In [4]:
vocab_file = "/content/drive/My Drive/DMML_Assignment_1/vocab.kos.txt"
docword_file = "/content/drive/My Drive/DMML_Assignment_1/docword.kos.txt"

Example 1 : K =2, F= 700

In [28]:
K = 2  # Set desired K-itemset size
F = 700  # Set minimum frequency threshold

find_frequent_itemsets(vocab_file, docword_file, K, F)


Finding Frequent 2-Itemsets with F=700...
Execution Time: 1.24 seconds
Total Frequent 2-Itemsets Found: 23

First 5 Frequent K-itemsets:
Itemset: ('administration', 'bush'), Count: 748
Itemset: ('bush', 'general'), Count: 1250
Itemset: ('bush', 'house'), Count: 873
Itemset: ('bush', 'kerry'), Count: 1195
Itemset: ('bush', 'president'), Count: 834


(1.2379562854766846, 23)

Example 2 : K =3, F= 700

In [29]:
find_frequent_itemsets(vocab_file, docword_file, 3, F = 700)


Finding Frequent 3-Itemsets with F=700...
Execution Time: 3.29 seconds
Total Frequent 3-Itemsets Found: 1

First 5 Frequent K-itemsets:
Itemset: ('bush', 'general', 'kerry'), Count: 926


(3.2868590354919434, 1)

Example 3 : K =5, F= 700

In [30]:
find_frequent_itemsets(vocab_file, docword_file, K = 5, F = 700)


Finding Frequent 5-Itemsets with F=700...
Execution Time: 103.57 seconds
Total Frequent 5-Itemsets Found: 0
No frequent itemsets found.


(103.57049608230591, 0)

### **Code 2:**
- **Purpose**: Implements the Apriori algorithm to efficiently find frequent $ K $-itemsets using an iterative candidate generation and pruning strategy.
- **Helper Functions**:
  1. **`read_vocab(vocab_file)`**:
     - Input: Vocabulary file path.
     - Output: Dictionary mapping word IDs to words.
  2. **`read_docword(docword_file)`**:
     - Input: Document-word file path.
     - Output: Dictionary mapping word IDs to sets of documents containing them.
  3. **`apriori(word_docs, K, F)`**:
     - Input: Word-document dictionary, $ K $ (itemset size), $ F $ (frequency threshold).
     - Output: Sorted list of frequent $ K $-itemsets with their counts.
     - Workflow:
       - Starts with frequent 1-itemsets.
       - Iteratively generates $ k $-itemsets from $ (k-1) $-itemsets using efficient pruning.
       - Counts support for candidates and filters infrequent ones.
  4. **`main(vocab_file, docword_file, K, F)`**:
     - Input: Vocabulary file, document-word file, $ K $, $ F $.
     - Output: Prints execution time, total frequent itemsets, and examples.

In [1]:
import time
from collections import defaultdict

def read_vocab(file_path):
    """ Read the vocabulary file and map word IDs to words. """
    vocab = {}
    with open(file_path, 'r') as f:
        for i, line in enumerate(f, start=1):
            vocab[i] = line.strip()
    return vocab

def read_docword(file_path):
    """ Read the document-word file and store word occurrences efficiently. """
    with open(file_path, 'r') as f:
        D = int(f.readline().strip())  # Number of documents
        W = int(f.readline().strip())  # Number of words
        NNZ = int(f.readline().strip())  # Nonzero entries

        word_docs = defaultdict(set)  # wordID -> set of documents containing it
        for _ in range(NNZ):
            doc_id, word_id, count = map(int, f.readline().strip().split())
            word_docs[word_id].add(doc_id)

    return word_docs

def apriori(word_docs, K, F):
    """ Optimized Apriori algorithm for large datasets. """

    # Step 1: Find frequent 1-itemsets
    freq_itemsets = { (word,): docs for word, docs in word_docs.items() if len(docs) >= F }

    # Step 2: Generate k-itemsets iteratively
    for k in range(2, K + 1):
        candidates = set()
        freq_keys = list(freq_itemsets.keys())  # List of current frequent itemsets

        # Generate candidate itemsets of size k using (k-1)-itemsets
        for i in range(len(freq_keys)):
            for j in range(i + 1, len(freq_keys)):
                a, b = freq_keys[i], freq_keys[j]

                # Merge only if first (k-2) elements are same (Efficient pruning)
                if a[:-1] == b[:-1]:
                    new_itemset = tuple(sorted(set(a) | set(b)))  # Union
                    if len(new_itemset) == k:
                        candidates.add(new_itemset)

        # Count support for candidate itemsets
        new_freq_itemsets = {}
        for c in candidates:
            intersect_docs = set.intersection(*(word_docs[word] for word in c))
            if len(intersect_docs) >= F:
                new_freq_itemsets[c] = intersect_docs

        # If no new frequent itemsets, break early
        if not new_freq_itemsets:
            return []

        freq_itemsets = new_freq_itemsets

    # Convert sets to counts for readability
    return sorted([(itemset, len(docs)) for itemset, docs in freq_itemsets.items()],
                  key=lambda x: x[1], reverse=True)

def main(vocab_file, docword_file, K, F):
    print("Reading dataset...")
    word_docs = read_docword(docword_file)
    vocab = read_vocab(vocab_file)

    print(f"Running Apriori for K={K}, F={F}")
    start_time = time.time()
    frequent_itemsets = apriori(word_docs, K, F)

    elapsed = time.time() - start_time

    print(f"\nTime taken: {elapsed:.2f} seconds")

    if not frequent_itemsets:
        print("\nNo itemsets found.")
    else:
        print(f"\nTotal Frequent K-itemsets Found: {len(frequent_itemsets)}")
        print("Frequent K-itemsets:")
        for itemset, count in frequent_itemsets:
            print(f"Itemset: {tuple(vocab[word] for word in itemset)}, Count: {count}")

Example 1 : K =2, F= 700

In [5]:
main(vocab_file, docword_file, K = 2, F = 700)

Reading dataset...
Running Apriori for K=2, F=700

Time taken: 0.05 seconds

Total Frequent K-itemsets Found: 23
Frequent K-itemsets:
Itemset: ('bush', 'general'), Count: 1250
Itemset: ('bush', 'kerry'), Count: 1195
Itemset: ('general', 'kerry'), Count: 1064
Itemset: ('bush', 'war'), Count: 994
Itemset: ('bush', 'democratic'), Count: 937
Itemset: ('democratic', 'kerry'), Count: 922
Itemset: ('democratic', 'primary'), Count: 894
Itemset: ('bush', 'poll'), Count: 876
Itemset: ('bush', 'house'), Count: 873
Itemset: ('kerry', 'poll'), Count: 845
Itemset: ('bush', 'president'), Count: 834
Itemset: ('bush', 'republicans'), Count: 794
Itemset: ('bush', 'democrats'), Count: 783
Itemset: ('bush', 'election'), Count: 766
Itemset: ('democratic', 'general'), Count: 763
Itemset: ('general', 'poll'), Count: 756
Itemset: ('democratic', 'poll'), Count: 755
Itemset: ('administration', 'bush'), Count: 748
Itemset: ('general', 'war'), Count: 745
Itemset: ('democratic', 'democrats'), Count: 740
Itemset: (

In [6]:
main(vocab_file, docword_file, K = 5, F = 700)

Reading dataset...
Running Apriori for K=5, F=700

Time taken: 0.06 seconds

No itemsets found.


In [7]:
main(vocab_file, docword_file, K = 5, F = 400)

Reading dataset...
Running Apriori for K=5, F=400

Time taken: 1.03 seconds

Total Frequent K-itemsets Found: 5
Frequent K-itemsets:
Itemset: ('bush', 'general', 'kerry', 'poll', 'polls'), Count: 437
Itemset: ('bush', 'democratic', 'democrats', 'general', 'republicans'), Count: 409
Itemset: ('bush', 'democratic', 'democrats', 'house', 'republicans'), Count: 403
Itemset: ('democratic', 'kerry', 'poll', 'polls', 'primary'), Count: 400
Itemset: ('bush', 'democratic', 'general', 'kerry', 'republicans'), Count: 400


Note that Code 2 is taking lesser time than Code 1. Naotable difference :

   - Code 1 uses **recursive brute-force generation** of $ K $-itemsets, which is computationally expensive.
   - Code 2 uses the **Apriori algorithm**, which prunes infrequent itemsets early, significantly reducing the number of candidates.
   - Code 2's iterative and pruning-based approach ensures faster execution times.

In summary, **Code 2 is better** because it is optimized for efficiency and performance, making it suitable for mining frequent itemsets in large datasets. Hence, we proceed with Code 2.